In [1]:
!git clone https://github.com/angel-one/smartapi-python.git

Cloning into 'smartapi-python'...
remote: Enumerating objects: 925, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 925 (delta 309), reused 273 (delta 264), pack-reused 528 (from 2)
Receiving objects: 100% (925/925), 233.87 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (550/550), done.


In [2]:
!cd /content/smartapi-python

In [ ]:
!pip install -r /content/smartapi-python/requirements_dev.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.8/480.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyotp
!pip install logzero
!pip install websocket-client

In [ ]:
!pip uninstall pycrypto
!pip install pycryptodome

In [ ]:
from datetime import datetime, timedelta
import time
from SmartApi import SmartConnect
import pyotp
import pytz
IST = pytz.timezone("Asia/Kolkata")

# Initialize API credentials
api_key = "aDk4LYUM"
client_id = "A61994896"
password = "3298"
totp_token = "FPELUPA7BHMESI3BPT34PAEPSY"

# Initialize API
def initialize_api():
    """Initialize the SmartConnect API and establish a session."""
    obj = SmartConnect(api_key=api_key)
    totp = pyotp.TOTP(totp_token).now()
    data = obj.generateSession(client_id, password, totp)
    refresh_token = data['data']['refreshToken']
    feed_token = obj.getfeedToken()
    return obj, refresh_token, feed_token

# Fetch Last Traded Price (LTP)
def fetch_ltp(obj, symboltoken):
    """Fetch the Last Traded Price (LTP) for the specified symbol."""
    try:
        ltp_data = obj.getMarketData(mode="LTP", exchangeTokens={"NSE": [symboltoken]})
        ltp_price = ltp_data['data']['fetched'][0]['ltp']
        return ltp_price
    except Exception as e:
        print(f"Error fetching LTP: {e}")
        return None

# Calculate Buy Parameters
def calculate_buy_params(ltp_data):
    """Calculate buy price, stop loss, target price, and quantity."""
    candle_9_40 = {
        'open': ltp_data[0],
        'high': max(ltp_data),
        'low': min(ltp_data),
        'close': ltp_data[-1]
    }
    print(f"9:45 AM Candle: {candle_9_40}")

    buy_price = candle_9_40['high']
    stop_loss_price = candle_9_40['low']
    target_price = buy_price + (3 * (buy_price - stop_loss_price))
    quantity = int(1000 / (buy_price - stop_loss_price))
    return buy_price, stop_loss_price, target_price, quantity

# Place Buy Order
def place_buy_order(obj, buy_price, quantity, tradingsymbol, symboltoken):
    quantity = 1
    """Place a buy order."""
    try:
        order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": symboltoken,
            "transactiontype": "BUY",
            "exchange": "NSE",
            "ordertype": "LIMIT",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": buy_price,
            "quantity": quantity,
        }
        order_id = obj.placeOrder(order_params)
        print(f"Buy order placed successfully. Order ID: {order_id}")
        return True
    except Exception as e:
        print(f"Error placing buy order: {e}")
        return False

# Place Sell Order
def place_sell_order(obj, sell_price, quantity, tradingsymbol, symboltoken):
    quantity = 1
    """Place a sell order."""
    try:
        order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": symboltoken,
            "transactiontype": "SELL",
            "exchange": "NSE",
            "ordertype": "LIMIT",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": sell_price,
            "quantity": quantity,
        }
        order_id = obj.placeOrder(order_params)
        print(f"Sell order placed successfully. Order ID: {order_id}")
        return True
    except Exception as e:
        print(f"Error placing sell order: {e}")
        return False

# Main Function
def main():
    print("Initializing API...")
    obj, refresh_token, feed_token = initialize_api()
    tradingsymbol = "BPCL-EQ"
    symboltoken = "526"

    print("Waiting for market to open...")
    market_open_time = datetime.now(IST).replace(hour=9, minute=15, second=0, microsecond=0)
    while datetime.now(IST) < market_open_time:
        time.sleep(1)

    print("Market open! Fetching LTP data...")

   # 9:40 AM to 9:45 AM IST
    candle_start_time = datetime.now(IST).replace(hour=9, minute=40, second=0, microsecond=0)
    candle_end_time = datetime.now(IST).replace(hour=9, minute=45, second=0, microsecond=0)
    ltp_data = []

    # Monitor LTP during the 9:40 to 9:45 window
    while datetime.now(IST) < candle_end_time:
        if datetime.now(IST) >= candle_start_time:
            ltp_price = fetch_ltp(obj, symboltoken)
            if ltp_price is not None:
                ltp_data.append(ltp_price)
            time.sleep(5)

    if not ltp_data:
        print("No LTP data available for the 9:40 candle.")
        return None

    # Calculate buy parameters
    buy_price, stop_loss_price, target_price, quantity = calculate_buy_params(ltp_data)
    print(f"Buy Price: {buy_price}, Target Price: {target_price}, Stop Loss Price: {stop_loss_price}, Quantity: {quantity}")

    # Monitor LTP data for buy condition
    bought = False
    while not bought:
        ltp_price = fetch_ltp(obj, symboltoken)
        if ltp_price and ltp_price >= buy_price:
            print(f"Buy condition met at LTP {ltp_price}. Placing buy order.")
            bought = place_buy_order(obj, buy_price, quantity, tradingsymbol, symboltoken)
        time.sleep(5)

    # Monitor LTP data for sell condition
    sold = False
    while not sold:
        ltp_price = fetch_ltp(obj, symboltoken)
        if ltp_price:
            if ltp_price <= stop_loss_price:
                print(f"Stop-loss hit at LTP {ltp_price}. Placing sell order.")
                sold = place_sell_order(obj, stop_loss_price, quantity, tradingsymbol, symboltoken)
            elif ltp_price >= target_price:
                print(f"Target hit at LTP {ltp_price}. Placing sell order.")
                sold = place_sell_order(obj, target_price, quantity, tradingsymbol, symboltoken)
        time.sleep(5)

    print("Trade complete.")

    # Terminate session
    obj.terminateSession(client_id)
    print("Session terminated.")

# Run the script
if __name__ == "__main__":
    main()
